<img src="https://jaipresentation.blob.core.windows.net/comm/jai_avatar.png" width="100" align="right"/>

# JAI - Trust your data

## Resolution: finding duplicates in your data
This is an example of how to use the resolution capabilities of JAI.

In this notebook we will use a subset of the [PC Games 2020](https://www.kaggle.com/jesneuman/pc-games) dataset to search for duplicate values in the 'Genres' column.

You can install JAI in your environment using `pip install jai-sdk`.

And you can read the docs [here](https://jai-sdk.readthedocs.io/en/stable/)!

If you have any comments or suggestions, feel free to contact us: jedis@jquant.com.br

*In no direction that we turn do we find ease or comfort. If we are honest and if we have the will to win we find only danger, hard work and iron resolution.* - Wendell Willkie

In [1]:
# JAI imports
from jai import Jai
from jai.processing import process_resolution

# I/O import
import pandas as pd

## Reading data

In [3]:
# it might take a few seconds to download this dataset (10MB) to your computer
DATASET_URL = "https://jaipresentation.blob.core.windows.net/data/games_jai.parquet"
df_games = pd.read_parquet(DATASET_URL)

In [4]:
# checking values in the Platform column
df_games["Platform"].value_counts()

PC                                                                 4977
macOS, PC                                                           517
Linux, macOS, PC                                                    472
Linux, PC, macOS                                                    453
PC, macOS                                                           433
                                                                   ... 
GameCube, PSP, PC, PlayStation 2, Xbox                                1
iOS, Android, PC, macOS, Nintendo Switch, Linux                       1
Nintendo Switch, macOS, iOS, Android, PC                              1
Xbox, GameCube, Linux, Classic Macintosh, PlayStation 2, PC           1
PC, PlayStation, PS Vita, Classic Macintosh, PSP, PlayStation 3       1
Name: Platform, Length: 1503, dtype: int64

We can see column `Platform` has some values that actually refer to the same thing (i.e., "Linux, macOS, PC" and "Linux, PC, macOS"). In other words, these are duplicate values.

## We can use JAI to find duplicates and standardize their values!

In [5]:
j = Jai("YOUR_AUTH_KEY")

### We call `resolution` passing a given `name` for the database and the `data` itself (i.e., column "Platform")

In [6]:
db_name = "games_resolution"
col = "Platform"
results = j.resolution(name=db_name, data=df_games[col], db_type="FastText")

Fiding threshold: 100%|██████████| 150/150 [00:00<00:00, 413367.67it/s]
/home/paulo/Code/jai-sdk/jai/processing.py:56: UserWarning: Threshold calculated automatically.
  warnings.warn("Threshold calculated automatically.")
Process: 100%|██████████| 1503/1503 [00:00<00:00, 20794.02it/s]
random sample size: 150
threshold: 1.779715197981172e-15




## OK, how do I interpret these results?
Each index in the `results` dataframe is related to an integer in the `resolution_id` column. This integer, in turn, is also an index! And it indicates which other sample is related to that index. So index 0 points to `resolution_id` 0, stating that sample number 0 is related to itself. No surprises there.

In [25]:
results.head(10)

,resolution_id
0,0
1,1
2,2
3,3
2,2
5,5
6,6
7,7
8,8
9,9


But what about indexes that are related to samples other than themselves? This will clear things up. Let's look at samples where the index DOES NOT match the `resolution_id` column:

In [8]:
res2 = results.copy()
res2["map_id"] = res2.index
res3 = res2.loc[res2["resolution_id"] != res2["map_id"]]
res3

,resolution_id,map_id
15,5,15
15,5,15
15,5,15
15,5,15
15,5,15
...,...,...
66,29,66
260,6,260
39,24,39
39,24,39


**Now we can see that samples 5 and 15 are actually referring to the same thing! Let's see if it checks out:**

In [9]:
print(f"Item 5: {df_games[col].iloc[5]}\nItem 15: {df_games[col].iloc[15]}")

Item 5: PlayStation 4, PC, Xbox One
Item 15: PC, PlayStation 4, Xbox One


It does check out! These samples are clearly a permutation of one another. 

## We can create groups of samples that refer to the same thing
This makes it easier for us to check if the output of `fill` is actually making any sense

In [10]:
# get groups
groups = dict()
for i in range(res3.shape[0]):
    fixed = res3["resolution_id"].iloc[i]
    moving = res3["map_id"].iloc[i]

    fixed_name = df_games[col].iloc[fixed]
    moving_name = df_games[col].iloc[moving]

    if fixed_name not in groups:
        groups[fixed_name] = {moving_name}
    else:
        groups[fixed_name].add(moving_name)

In [11]:
# get all platforms that correspond to 'Linux, PC, macOS, PlayStation 4' 
groups["Linux, PC, macOS, PlayStation 4"]

{'Linux, macOS, PC, PlayStation 4',
 'PC, macOS, Linux, PlayStation 4',
 'macOS, Linux, PC, PlayStation 4',
 'macOS, PC, Linux, PlayStation 4'}

The cell above shows all values that belong to the same group (`"Linux, PC, macOS, PlayStation 4"`). They are all permutations of one another and are indeed duplicates!

Finally, we can standardize column `Platform` mapping permutations to a single, consistent value using the `results` dataframe.

In [28]:
results.head(10)

,resolution_id
0,0
1,1
2,2
3,3
2,2
5,5
6,6
7,7
8,8
9,9


In [29]:
new_platform = [df_games[col].iloc[results["resolution_id"].iloc[item]] for item in range(results.shape[0])]

In [43]:
new_platform_col = "Platform resolved"
df_games_resolved = df_games.copy()
df_games_resolved[new_platform_col] = new_platform

In [44]:
print(f"[df_games]\nItem 5: {df_games[col].iloc[5]}\nItem 15: {df_games[col].iloc[15]}")
print(f"\n[df_games_resolved]\nItem 5: {df_games_resolved[new_platform_col].iloc[5]}\nItem 15: {df_games_resolved[new_platform_col].iloc[15]}")

[df_games]
Item 5: PlayStation 4, PC, Xbox One
Item 15: PC, PlayStation 4, Xbox One

[df_games_resolved]
Item 5: PlayStation 4, PC, Xbox One
Item 15: PlayStation 4, PC, Xbox One
